In [1]:
import consul
import sys
import uuid

In [ ]:
def register_service(service_name, service_ip, service_port, consul_ip, consul_port):
    consul_client = consul.Consul(host=consul_ip, port=consul_port)

    service_id = f"{service_name}-{str(uuid.uuid4())[:4]}"

    consul_client.agent.service.register(
    name=service_name,
    service_id=service_id,
    address=service_ip,
    port=service_port,
    check=consul.Check.http(
        url=f"http://{service_ip}:{consul_ip}/health",
        interval="10s",
        timeout="1s",
        deregister="10m"
    )
)

In [2]:
SERVICE_NAME = sys.argv[0][:-3]
SERVICE_ID = f"{SERVICE_NAME}-{str(uuid.uuid4())[:4]}"

CONSUL_IP = "localhost"
CONSUL_PORT = 8500

SERVICE_IP = "host.docker.internal"
SERVICE_PORT = 5000

In [3]:
c = consul.Consul(host=CONSUL_IP, port=CONSUL_PORT)
try:
    leader = c.status.leader()
    print(f"Consul is available. Leader: {leader}")
except Exception as e:
    print(f"Failed to connect to Consul: {e}")

Consul is available. Leader: 127.0.0.1:8300


In [ ]:
register_service(SERVICE_NAME, SERVICE_IP, SERVICE_PORT, CONSUL_IP, CONSUL_PORT)

### Get all services

In [7]:
services = c.agent.services()
for service_id, service_info in services.items():
    print(f"Service ID: {service_id}")
    print(f"Service Name: {service_info['Service']}")
    print(f"Service Address (IP): {service_info['Address']}")
    print(f"Service Port: {service_info['Port']}")
    print("-" * 30)

Service ID: facade_service.py-ccf6
Service Name: facade_service.py
Service Address (IP): 127.0.0.1
Service Port: 5004
------------------------------
Service ID: logging_service.py-8f0e
Service Name: logging_service.py
Service Address (IP): 127.0.0.1
Service Port: 5002
------------------------------
Service ID: logging_service.py-94a6
Service Name: logging_service.py
Service Address (IP): 127.0.0.1
Service Port: 5001
------------------------------
Service ID: logging_service.py-ebc6
Service Name: logging_service.py
Service Address (IP): 127.0.0.1
Service Port: 5003
------------------------------
Service ID: messages_service.py-29a5
Service Name: messages_service.py
Service Address (IP): 127.0.0.1
Service Port: 5007
------------------------------


In [31]:
services = c.agent.services()
def get_service_ips_consul(service_name: str):
    service_name = service_name.replace("-", "_")
    result_ips = []

    for service_id, service_info in services.items():
        if service_name == service_info["Service"][:-3]:
            result_ips.append(f"{service_info['Address']}:{service_info['Port']}")

    return result_ips

In [33]:
print(get_service_ips_consul("messages-service"))


['127.0.0.1:5007']


### Set a key

In [ ]:
# Set a key
key = 'a'
value = 'b'

c.kv.put(key, value)

print(f"Key '{key}' set successfully!")

### Get a key

In [ ]:
index, data = c.kv.get('a')

if data is not None:
    value = data['Value'].decode('utf-8')  # Must decode from bytes
    print(f"Key value: {value}")
else:
    print("Key not found.")
